# Missing link prediction - Kaggle competition

In [2]:
# Libraries

In [6]:
import pandas as pd
import numpy as np
import networkx as nx

In [23]:
import random
import igraph
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import csv

In [7]:
# Read data

In [24]:
with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

In [26]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

In [35]:
n_info = pd.read_csv('node_information.csv',header=None)

In [39]:
colnames = ['id','year','title','author','journal','abstract']

In [41]:
n_info.columns = colnames

In [42]:
n_info

,id,year,title,author,journal,abstract
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...
...,...,...,...,...,...,...
27765,9912289,2002,gauge fixing in the chain by chain method,"A Shirzad, F Loran",NaN,in a recent work we showed that for a hamilton...
27766,9912290,2000,shuffling quantum field theory,Dirk Kreimer,Lett.Math.Phys.,we discuss shuffle identities between feynman ...
27767,9912291,1999,small object limit of casimir effect and the s...,"O. Kenneth, S. Nussinov",Phys.Rev.,we show a simple way of deriving the casimir p...
27768,9912292,1999,1 4 pbgs and superparticle actions,"F.Delduc, E. Ivanov, S. Krivonos",NaN,karpacz poland september 21-25 1999 we constru...


# Feature Engineering

## Doc2vec

## Node2vec

## GATConv layer

## Basic Graph metrics

In [32]:
node_info[:3]

[['1001',
  '2000',
  'compactification geometry and duality',
  'Paul S. Aspinwall',
  '',
  'these are notes based on lectures given at tasi99 we review the geometry of the moduli space of n 2 theories in four dimensions from the point of view of superstring compactification the cases of a type iia or type iib string compactified on a calabi-yau threefold and the heterotic string compactified on k3xt2 are each considered in detail we pay specific attention to the differences between n 2 theories and n 2 theories the moduli spaces of vector multiplets and the moduli spaces of hypermultiplets are reviewed in the case of hypermultiplets this review is limited by the poor state of our current understanding some peculiarities such as mixed instantons and the non-existence of a universal hypermultiplet are discussed'],
 ['1002',
  '2000',
  'domain walls and massive gauged supergravity potentials',
  'M. Cvetic, H. Lu, C.N. Pope',
  'Class.Quant.Grav.',
  'we point out that massive gauged 

In [13]:
network = pd.read_csv('node_information.csv',header=None)

In [14]:
network

,0,1,2,3,4,5
0,1001,2000,compactification geometry and duality,Paul S. Aspinwall,NaN,these are notes based on lectures given at tas...
1,1002,2000,domain walls and massive gauged supergravity p...,"M. Cvetic, H. Lu, C.N. Pope",Class.Quant.Grav.,we point out that massive gauged supergravity ...
2,1003,2000,comment on metric fluctuations in brane worlds,"Y.S. Myung, Gungwon Kang",NaN,recently ivanov and volovich hep-th 9912242 cl...
3,1004,2000,moving mirrors and thermodynamic paradoxes,Adam D. Helfer,Phys.Rev.,quantum fields responding to moving mirrors ha...
4,1005,2000,bundles of chiral blocks and boundary conditio...,"J. Fuchs, C. Schweigert",NaN,proceedings of lie iii clausthal july 1999 var...
...,...,...,...,...,...,...
27765,9912289,2002,gauge fixing in the chain by chain method,"A Shirzad, F Loran",NaN,in a recent work we showed that for a hamilton...
27766,9912290,2000,shuffling quantum field theory,Dirk Kreimer,Lett.Math.Phys.,we discuss shuffle identities between feynman ...
27767,9912291,1999,small object limit of casimir effect and the s...,"O. Kenneth, S. Nussinov",Phys.Rev.,we show a simple way of deriving the casimir p...
27768,9912292,1999,1 4 pbgs and superparticle actions,"F.Delduc, E. Ivanov, S. Krivonos",NaN,karpacz poland september 21-25 1999 we constru...


In [44]:
# fuzzywuzzy forremoving duplicates

In [3]:
import argparse
from os import path

import numpy as np
import torch
import torch.nn.functional as F
from dgl import batch
from dgl.data.ppi import LegacyPPIDataset
from dgl.nn.pytorch import GraphConv
from dgl.nn.pytorch.conv import GATConv
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


MODEL_STATE_FILE = path.join(path.dirname(path.abspath(__file__)), "model_state.pth")


import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


def train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset):
    train_score_list = []
    test_score_list = []
    loss_data_list = []
    loss_test_list = []
    epoch_between_two_scores = 1
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                           factor=0.5, patience=5, verbose=True, 
                                                           min_lr=1e-5, eps=1e-08)
    early_stopping = EarlyStopping(patience=30, verbose=False)
    for epoch in range(args.epochs):
        # train
        model.train()
        losses = []
        for batch, data in enumerate(train_dataloader):
            subgraph, features, labels = data
            features = features.to(device)
            labels = labels.to(device)
            logits = model(subgraph, features.float())
            loss = loss_fcn(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        loss_data = np.array(losses).mean()
        loss_data_list.append(loss_data)
        
        loss_mean = []
        for batch, test_data in enumerate(test_dataset):
            subgraph, features, labels = test_data
            features = torch.tensor(features).to(device)
            labels = torch.tensor(labels).to(device)
            _, loss = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
            loss_mean.append(loss)
        mean_test_loss = np.array(loss_mean).mean()
        loss_test_list.append(mean_test_loss)
        scheduler.step(mean_test_loss)
        early_stopping(mean_test_loss, model)
            
        print("Epoch {:05d} | Train Loss: {:.4f} | Test Loss: {:.4f}".format(epoch + 1, loss_data, mean_test_loss))
       
        # validate
        if epoch % epoch_between_two_scores == 0:
            train_scores = []
            for batch, data in enumerate(train_dataloader):
                subgraph, features, labels = data
                features = features.to(device)
                labels = labels.to(device)
                train_score, _ = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
                train_scores.append(train_score) 
            mean_train_score = np.array(train_scores).mean()
            train_score_list.append(mean_train_score)
            
            test_scores = []
            for batch, test_data in enumerate(test_dataset):
                subgraph, features, labels = test_data
                features = torch.tensor(features).to(device)
                labels = torch.tensor(labels).to(device)
                test_score, _ = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
                test_scores.append(test_score)
            mean_test_score = np.array(test_scores).mean()
            test_score_list.append(mean_test_score)
            print("   F1-Score | Train       {:.2%} | Test       {:.2%} ".format(mean_train_score, mean_test_score))
        
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
    history = {
        'score':{
            'train': train_score_list,
            'test': test_score_list,
            'epoch_between_two_scores': epoch_between_two_scores,
        },
        'loss':{
            'train': loss_data_list,
        }
    }
    return history


def test(model, loss_fcn, device, test_dataloader):
    test_scores = []
    for batch, test_data in enumerate(test_dataloader):
        subgraph, features, labels = test_data
        features = features.to(device)
        labels = labels.to(device)
        test_scores.append(evaluate(features, model, subgraph, labels.float(), loss_fcn)[0])
    mean_scores = np.array(test_scores).mean()
    print("F1-Score: {:.4f}".format(np.array(test_scores).mean()))
    return mean_scores


def evaluate(features, model, subgraph, labels, loss_fcn):
    with torch.no_grad():
        model.eval()
        model.graph = subgraph
        output = model(subgraph, features.float())
        loss_data = loss_fcn(output, labels.float())
        predict = np.where(output.data.cpu().numpy() >= 0.5, 1, 0)
        score = f1_score(labels.data.cpu().numpy(), predict, average="micro")
        return score, loss_data.item()


def collate_fn(sample):
    graphs, features, labels = map(list, zip(*sample))
    graph = batch(graphs)
    features = torch.from_numpy(np.concatenate(features))
    labels = torch.from_numpy(np.concatenate(labels))
    return graph, features, labels

def plot_score_fc(history):
    step = history['score']['epoch_between_two_scores']
    epoch_list = range(0, len(history['score']['train']) * step, step)
    
    fig = plt.figure(figsize = (13, 6))
    ax1 = fig.add_subplot(1,1,1)
    
    train_score = history['score']['train']
    ax1.plot(epoch_list, train_score)

    test_score = history['score']['test']
    ax1.plot(epoch_list, test_score)
    
    ax1.legend(['train score',' test score'])
    ax1.set_xlabel('Epochs', size = 15)
    ax1.set_ylabel('F1-score', size = 15)
    ax1.set_title('Score of the train and test set each {step} epochs'.format(step = step), size = 20)

    ax1.set_xlim(0, epoch_list[-1])
    ax1.set_ylim(0,1)

    ax1.set_yticks([value/10 for value in range(1, 11)])
    fig.savefig('score.png')

class Model(nn.Module):

    def __init__(self, n_layers, input_size, hidden_size, output_size, nonlinearity):
        super().__init__()
        self.gat1 = GATConv(in_feats=input_size,
                                    out_feats=256,
                                    num_heads=4,
                                   activation = F.elu
                                  )
        self.gat2 = GATConv(in_feats=256 * 4,
                                    out_feats=256,
                                    num_heads=4,
                                   activation = F.elu
                                  )
        self.gat4 = GATConv(in_feats=256 * 4,
                                    out_feats=256,
                                    num_heads=4,
                                   activation = F.elu
                                  )
        self.gat3 = GATConv(in_feats=256 * 4,
                                    out_feats=121,
                                    num_heads=6,
                                   activation= F.elu
                                  )
        self.flatten1 = nn.Flatten()
        self.flatten2 = nn.Flatten()
        self.linear0 = nn.Linear(input_size, 256 * 4)
        self.linear1 = nn.Linear(256 * 4, 256 * 4)
        self.linear2 = nn.Linear(121 * 3, 121)
        self.linear3 = nn.Linear(256 * 4, 121)
        self.dropout = nn.Dropout(0.3)
    def forward(self, subgraph,  inputs):
        hidden = self.gat1(subgraph, inputs)
        hidden = self.flatten1(hidden)
        hidden2 = hidden
        hidden = (hidden + F.relu(self.dropout(self.linear0(inputs)))) / 2
        
        shortcut = hidden
        hidden = self.dropout(hidden)
        hidden = self.gat2(subgraph, hidden)
        hidden = self.flatten2(hidden)
        
        hidden2 = self.dropout(hidden2)
        hidden2 = self.gat4(subgraph, hidden2)
        hidden2 = self.flatten2(hidden2)
        
        hidden = (hidden +hidden2+ F.relu(self.dropout(self.linear1(shortcut)))) / 3
        shortcut = hidden 
        
        hidden = self.dropout(hidden)
        hidden = self.gat3(subgraph,hidden)
        hidden_mean = (hidden.mean(dim = 1) + F.relu(self.linear3(shortcut))) / 2
        hidden_max = (hidden.max(dim = 1).values + F.relu(self.linear3(shortcut))) / 2
        hidden_min = (hidden.min(dim = 1).values + F.relu(self.linear3(shortcut))) / 2
        hidden = torch.cat([hidden_mean, hidden_max, hidden_min], dim = 1)
        hidden = torch.sigmoid(self.linear2(hidden))
        return hidden

def main(args):
    # create the dataset
    train_dataset, test_dataset = LegacyPPIDataset(mode="train"), LegacyPPIDataset(mode="test")
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    n_features, n_classes = train_dataset.features.shape[1], train_dataset.labels.shape[1]

    # create the model, loss function and optimizer
    device = torch.device("cpu" if args.gpu < 0 else "cuda:" + str(args.gpu))
    model = Model(n_layers=None, input_size=n_features,hidden_size=256, output_size=n_classes, nonlinearity=F.elu).to(device)
    loss_fcn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

    # train and test
    if args.mode == "train":
        history = train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset)
        torch.save(model.state_dict(), MODEL_STATE_FILE)
        if args.verbose:
            plot_score_fc(history)
        return history, test(model, loss_fcn, device, test_dataloader)
    model.load_state_dict(torch.load(MODEL_STATE_FILE, map_location=lambda storage, loc: storage))

    return test(model, loss_fcn, device, test_dataloader)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode",  choices=["train", "test"], default="test")
    parser.add_argument("--gpu", type=int, default=-1, help="GPU to use. Set -1 to use CPU.")
    parser.add_argument("--epochs", type=int, default=400)
    parser.add_argument("--batch-size", type=int, default=2)
    parser.add_argument("--verbose", type=bool, default= False)
    args = parser.parse_args()
    main(args)



NameError: name '__file__' is not defined